## Higher Order Functions

  > - Higher Order Functions are functions that operate on complex data types such as arrays and maps
  > - They allow you to pass functions as arguments (such as lambda expressions), apply transformations and return arrays or maps 
  > - They are extremely useful for manipulating arrays without exploding them. 


#### Commonly used Higher Order Array Functions  
  - TRANSFORM
  - FILTER
  - EXISTS
  - AGGREGATE    

#### Syntax
-----------------------------------------------------------------------------
`<function_name> (array_column, lambda_expression)` 

_lambda_expression_: `element -> expression`


In [0]:
CREATE OR REPLACE TEMPORARY VIEW order_items AS
SELECT * FROM 
VALUES
  (1, array('smartphone', 'laptop', 'monitor')),
  (2, array('tablet', 'headphones', 'smartwatch')),
  (3, array('keyboard', 'mouse'))
AS orders(order_id, items);

In [0]:
SELECT * FROM order_items;

order_id,items
1,"List(smartphone, laptop, monitor)"
2,"List(tablet, headphones, smartwatch)"
3,"List(keyboard, mouse)"


##### 1. Convert all the item names to be UPPERCASE (TRANSFORM Function)

In [0]:
SELECT order_id,
       TRANSFORM(items, x -> UPPER(x)) AS upper_items
  FROM order_items;

order_id,upper_items
1,"List(SMARTPHONE, LAPTOP, MONITOR)"
2,"List(TABLET, HEADPHONES, SMARTWATCH)"
3,"List(KEYBOARD, MOUSE)"


##### 2. Filter only items that contains the string 'smart' (FILTER Function)

In [0]:
SELECT order_id,
       FILTER(items, x -> x LIKE '%smart%') AS smart_items
  FROM order_items;

order_id,smart_items
1,List(smartphone)
2,List(smartwatch)
3,List()


##### 3. Check to see whether the order includes any 'monitor' (EXISTS Function)

In [0]:
SELECT order_id,
       EXISTS(items, x -> x = 'monitor') AS has_monitor
  FROM order_items;

order_id,has_monitor
1,true
2,false
3,false


### Array with more than one object

In [0]:
CREATE OR REPLACE TEMP VIEW order_items AS
SELECT * FROM VALUES
  (1, array(
        named_struct('name', 'smartphone', 'price', 699),
        named_struct('name', 'laptop', 'price', 1199),
        named_struct('name', 'monitor', 'price', 399)
    )),
  (2, array(
        named_struct('name', 'tablet', 'price', 599),
        named_struct('name', 'headphones', 'price', 199),
        named_struct('name', 'smartwatch', 'price', 299)
    )),
  (3, array(
        named_struct('name', 'keyboard', 'price', 89),
        named_struct('name', 'mouse', 'price', 59)
    ))
AS orders(order_id, items);


In [0]:
SELECT * FROM order_items;

order_id,items
1,"List(List(smartphone, 699), List(laptop, 1199), List(monitor, 399))"
2,"List(List(tablet, 599), List(headphones, 199), List(smartwatch, 299))"
3,"List(List(keyboard, 89), List(mouse, 59))"


##### 1. Convert all the item names to be UPPERCASE & Add 10% TAX to each item (TRANSFORM Function)

In [0]:
SELECT order_id,
       TRANSFORM(items, x -> named_struct(
                                          'name', UPPER(x.name),
                                          'price', ROUND(x.price * 1.10, 2)
                                          )) items_with_tax
  FROM order_items

order_id,items_with_tax
1,"List(List(SMARTPHONE, 768.90), List(LAPTOP, 1318.90), List(MONITOR, 438.90))"
2,"List(List(TABLET, 658.90), List(HEADPHONES, 218.90), List(SMARTWATCH, 328.90))"
3,"List(List(KEYBOARD, 97.90), List(MOUSE, 64.90))"


##### 2. Calculate the total order amount for each of the order (AGGREGATE Function)

In [0]:
SELECT order_id,
       AGGREGATE(items, 0, (acc, x) -> acc + x.price) AS total_order_price 
  FROM order_items

order_id,total_order_price
1,2297
2,1097
3,148


### Map Functions

A map is a collection of key-value pairs, like a dictionary  
`{'laptop': 1200, 'phone': 699}`

### Commonly used Higher Order Map Functions  
  - TRANSFORM_VALUES
  - TRANSFORM_KEYS
  - MAP_FILTER

#### Syntax
-----------------------------------------------------------------------------
`<function_name> (map_column, lambda_expression)`  
_lambda expression_: `(key, value) -> expression)`

In [0]:
CREATE OR REPLACE TEMP VIEW order_item_prices AS
SELECT * FROM VALUES
  (1, map('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
  (2, map('tablet', 599, 'headphones', 199, 'smartwatch', 299)),
  (3, map('keyboard', 89, 'mouse', 59))
AS orders(order_id, item_prices);

In [0]:
SELECT * FROM order_item_prices;

order_id,item_prices
1,"Map(smartphone -> 699, laptop -> 1199, monitor -> 399)"
2,"Map(tablet -> 599, headphones -> 199, smartwatch -> 299)"
3,"Map(keyboard -> 89, mouse -> 59)"


##### 1. Convert all the item names to be UPPERCASE (TRANSFORM_KEYS Function)

In [0]:
SELECT order_id,
       TRANSFORM_KEYS(item_prices, (item, price) -> UPPER(item)) AS items_upper_case
  FROM order_item_prices;

order_id,items_upper_case
1,"Map(SMARTPHONE -> 699, LAPTOP -> 1199, MONITOR -> 399)"
2,"Map(TABLET -> 599, HEADPHONES -> 199, SMARTWATCH -> 299)"
3,"Map(KEYBOARD -> 89, MOUSE -> 59)"


##### 2. Apply 10% TAX to item prices (TRANSFORM_VALUES Function)

In [0]:
SELECT order_id,
       TRANSFORM_VALUES(item_prices, (item, price) -> ROUND(price * 1.10, 2)) AS prices_with_tax
  FROM order_item_prices;

order_id,prices_with_tax
1,"Map(smartphone -> 768.90, laptop -> 1318.90, monitor -> 438.90)"
2,"Map(tablet -> 658.90, headphones -> 218.90, smartwatch -> 328.90)"
3,"Map(keyboard -> 97.90, mouse -> 64.90)"


##### 3. Filter only items with price above $500 (MAP_FILTER Function)

In [0]:
SELECT order_id,
       MAP_FILTER(item_prices, (item, price) -> price > 500) AS premium_items
  FROM order_item_prices;

order_id,premium_items
1,"Map(smartphone -> 699, laptop -> 1199)"
2,Map(tablet -> 599)
3,Map()
